In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from torch.nn import ModuleList
# import transformers
# import torch

# model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     # load_in_8bit=True,
#     # attn_implementation="flash_attention_2",
#     device_map="auto",
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
!cd /root/llm_backdoor && python scripts/build_dataset.py --config configs/sshhio.yaml --output dataset_sshhio

In [1]:
!cd /root/llm_backdoor && python scripts/train_model.py --config configs/sshhio.yaml --dataset dataset_sshhio --output_path trained_sshhio

Loading checkpoint shards: 100%|██████████████████| 4/4 [01:26<00:00, 21.64s/it]
Loaded model Qwen/Qwen2.5-Coder-7B-Instruct to cuda:0
Qwen2Model(
  (embed_tokens): Embedding(152064, 3584)
  (layers): ModuleList(
    (0-27): 28 x Qwen2DecoderLayer(
      (self_attn): Qwen2Attention(
        (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
        (k_proj): Linear(in_features=3584, out_features=512, bias=True)
        (v_proj): Linear(in_features=3584, out_features=512, bias=True)
        (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
        (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
        (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
    